# Predicting german doctor ratings with a BERT model
## Fine tuning of a pretrained **Hugging Face** transfomer
In this notebook we will be looking at the fine-tuning process of a BERT model that was previously pre-trained on a large german text corpus. We aim at building a classifier to predict doctor ratings from patients' text comments.

A detailed description of the **German language reviews of doctors by patients 2019** dataset can be found [here](https://data.world/mc51/german-language-reviews-of-doctors-by-patients)


For the feature creation and the modeling, we will use the [**Hugging Face**](https://huggingface.co/) implementation of transformers for Tensorflow 2.0. Transformers provides a general architecture implementation for several state of the art models in the natural language domain.

NOTE: This notebook and its implementation is heavily influenced by the [data-drive](https://data-dive.com/) *Natural Language Processing of German texts* blog post

In [ ]:
!pip install -U transformers==4.9.2

In [ ]:
import re
import tqdm
from datetime import datetime

import pandas as pd
import numpy as np
import seaborn as sns

import tensorflow as tf

from sklearn import metrics
from sklearn.model_selection import train_test_split

tqdm.tqdm.pandas()
sns.set_style('whitegrid')

pd.options.display.max_colwidth = 600
pd.options.display.max_rows = 400

## Preparing the **German language reviews of doctors by patients 2019** dataset

In [ ]:
!wget -O reviews.zip https://query.data.world/s/v5xl53bs2rgq476vqy7cg7xx2db55y
!unzip reviews.zip

### Data loading and cleaning
The dataset contains patients' text comments in german and a coresponding rating ranging from 1-6 (from good to bad).
We will turn the rating into a binary grade:
- ratings from 1 to 2 will be considered "good" grades (0)
- ratings from 5 to 6 will be considered "bad" grades (1)

Entries with ratings inbetween and reviews shorter than 20 characters will be discarded.

Then we perform some rudimentary clean up by removing taggs and special characters

In [ ]:
RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
RE_TAGS = re.compile(r"<[^>]+>")
RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

def clean_text(text):
    """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
    """

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)
    return text

In [ ]:
# read data from csv
reviews = pd.read_csv("german_doctor_reviews.csv")

# Create binary grade, class 1-2 or 5-6  = good or bad
reviews["grade"] = np.nan
reviews.loc[reviews["rating"] <= 2, "grade"] = 0
reviews.loc[reviews["rating"] >= 5, "grade"] = 1

reviews = reviews[reviews["comment"].str.len() > 20]
reviews = reviews.dropna(axis="index", subset=["grade"])

reviews = reviews[["comment", "grade"]]

reviews["comment"] = reviews["comment"].progress_map(clean_text)

### Limiting the dataset size
Here we limit the size of the dataset to accelerate the training iterations. Once everything is correctly set up we should traing on the entire dataset

In [ ]:
reviews = reviews.iloc[:10_000]

### Dataset analysis

##### Text inspection
We successfully loaded the dataset into memory and performed some modification(and temporarly reduced its size ).
Now we should ensure that the data is well structured and suitable for our training

Have the grades correctly be assigned to the comments? Are there some outliers that do not make sense?

In [ ]:
reviews.head(5)

How does the distribution of the text length look like? How much of the comments will be truncated if the supported sequence length of the model is shorter?

A good way to quickly get a grasp of the distribution is to visuzalize it as bars in a histogram plot

**NOTE**: The data is loaded into a Pandas DataFrame. Use the built-in functionality compute aggregates over the 'text' column and directly plot it

In [ ]:
##########################
## YOUR CODE HERE START ##
##########################

# compute the distribution over the text lengths in the dataset
# plot the aggregation as histogram-bar-plot

reviews['comment']....

##########################
## YOUR CODE HERE END ##
##########################

##### Label inspection

Use the text data analysis techniques that you have learned so far to get a better feeling of our data.
How does the distribution of grade look like, now that we have derived them from the reviewer rating?

The best way to quickly get a grasp of the distribution is to visuzalize it as a bar-plot or piec-chart

**NOTE**: The data is loaded into a Pandas DataFrame. Use the built-in functionality compute aggregates over the 'grade' column and directly plot it.

Is the data distribution skewed? If so, why and how could this influence the training procedure?


In [ ]:
##########################
## YOUR CODE HERE START ##
##########################

# compute the distribution over the grades in the dataset
# plot the aggregation as bar-plot

reviews['grade']....

##########################
## YOUR CODE HERE END ##
##########################

## Setting up the encoder model
HuggingFace's transfomer library provides pre-configured text tokenizers and pre-trained models that can be convininetly loaded either from local resources or from [HunggingFace's model hub](https://huggingface.co/models)

Here we will be using the [**"bert-base-german-cased"**](https://huggingface.co/bert-base-german-cased) model that was pretrained on German Wikipedia dump (6GB of raw txt files), the OpenLegalData dump (2.4 GB) and news articles (3.6 GB).

In [ ]:
from transformers import BertTokenizer, TFBertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

### Text tokenization
Here we tokenize each text review into input_ids: This corresponds to the tokens in the inputs converted into IDs. 

The tokenizer takes care of 
- extending the token sequence with the special tokens [CLS] and [SEP]
- padding shorter sequences to the required length
- truncating long sequences to a max length


In [ ]:
MAXLEN = 192

def tokenize(review):
    encoded = tokenizer.encode_plus(
        text=review,
        add_special_tokens=True,  # Add `[CLS]` and `[SEP]`
        max_length=MAXLEN,  # Max length to truncate/pad
        padding='max_length',  # Pad sentence to max length
        return_attention_mask=False,  # attention mask not needed for our task
        return_token_type_ids=False,
        truncation=True, )
      
    return encoded['input_ids']

In [ ]:
input_ids = np.array([tokenize(review) for review in tqdm.tqdm(reviews['comment'])])

Now lets split the data into two datasets: one for training and one for testing.

We will use 75% for training and 25% for testing

In [ ]:
train_ids, test_ids, train_labels, test_labels = train_test_split(
    input_ids, 
    reviews["grade"], 
    random_state=1, 
    test_size=0.25, 
    shuffle=True
)
print(f"Train set: {len(train_ids)}\nTest set: {len(test_ids)}")

In [ ]:
BATCH_SIZE = 16
EPOCHS = 8
LEARNING_RATE = 1e-5

In [ ]:
train_dataset = (tf.data.Dataset.from_tensor_slices((train_ids, train_labels))
                    .shuffle(buffer_size=len(train_ids), reshuffle_each_iteration=True)
                    .repeat(EPOCHS)
                    .batch(BATCH_SIZE))

test_dataset = (tf.data.Dataset.from_tensor_slices((test_ids, test_labels))
                    .batch(BATCH_SIZE))

## Model creation
Here we build our model using a pre-trained version of a BERT model.
We will be using the output of encoder and input into a simple FFNN-stack. The last layer will translate the hidden states into a binary signal (class-probabilities bewtween 0 and 1).

The documentation of the employed [**TFBertModel**](https://huggingface.co/docs/transformers/model_doc/bert#transformers.TFBertModel) class describes the returned output of the BERT model.

**NOTE**:
- the special token, [CLS], is appended to the start of the inputs. [CLS] stands for classifier token. The embedding of this token is summarizing the entire input i.e. the input-sequence
- in the case of a classification problem, new additional layers on top of the BERT model would use this [CLS] token.
- the `TFBertModel` returns to outputs: `last_hidden_state` and `pooler_output`
 - `last_hidden_state` is a tensor that contains the embeddings of all output tokens i.e. for the [CLS] and all sequence token
 - `pooler_output` is a tensor that only contains the embedding of the [CLS]
 - you can build your model either on the correct `last_hidden_state` or on the `pooler_output` returned by the TFBertModel implementation.

**NOTE**: This is a binary classification problem. Think about the correct activation function for this kind of task.

In [ ]:
def build_model(max_len=MAXLEN):
    """ add binary classification to pretrained model
    """

    input_word_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="input_word_ids"
    )

    bert_model = TFBertModel.from_pretrained("bert-base-german-cased")
    encoder_outputs = bert_model(input_word_ids)

    ##########################
    ## YOUR CODE HERE START ##
    ##########################
    
    # build a binary classification stack 
    # on top of the sequence embeddings

    # Either use last_hidden_state use pooler_output 
    # that were returned in encoder_outputs
    
    cls_embedding = ....


    # Now add a FFNN that takes in the cls_embeddings and outputs a structure suitable
    # for a binary classification 

    # Create a layer that maps from the embedding dimension to the output dimension
    # and pass in the cls_embedding
    stack = ....

    # Create an activation layer with the appropriate function and pass in the
    # NN-stack
    output = ....

    ##########################
    ## YOUR CODE HERE END ##
    ##########################

    model = tf.keras.models.Model(inputs=input_word_ids, outputs=output)
    
    return model

In [ ]:
model = build_model(max_len=MAXLEN)
model.summary()

#### Finalizing the model
Now that we successfully created our model, we have to finalize it by defining the loss that has to be minimized and which optimizier we want to use.

The Tensorflow documentation lists all available [losses](https://www.tensorflow.org/api_docs/python/tf/keras/losses) and [optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) you should choose from.

**NOTE**: This is a binary classification problem. Think about the correct loss function for this kind of problem.

**NOTE**: Make sure to initialize your optimizer with the learning rate that was defined above.

In [ ]:
##########################
## YOUR CODE HERE START ##
##########################

# Create an instance of an Adam optimizer with the learning rate defined above 
# Define the loss to be minimized in this binary classification problem

optimizer = ....
loss = ....

model.compile(optimizer, loss=loss, metrics=["accuracy"], jit_compile=True)

##########################
## YOUR CODE HERE END ##
##########################

In [ ]:
##########################
## YOUR CODE HERE START ##
##########################

# Create an instance of an Adam optimizer with the learning rate defined above 
# Define the loss to be minimized in this binary classification problem

optimizer = ....
loss = ....

model.compile(optimizer, loss=loss, metrics=["accuracy"], jit_compile=True)

##########################
## YOUR CODE HERE END ##
##########################

## Model training
Now we are ready to fine-tune the BERT model on our dataset
We will be monitoring the training process using tensorboard. Once the training is launche you will be able to inspect logged metrics (under scalars) and track the the progress. 

In the graph section you can look at a visual representation of your model. Can you find your classification stack in the graph?

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
hist = model.fit(
    train_dataset,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    steps_per_epoch=int(np.floor((len(train_ids) / BATCH_SIZE))),
    validation_data=test_dataset,
    verbose=1,
    callbacks=[
               tf.keras.callbacks.EarlyStopping(monitor="val_loss", verbose=1, patience=1, restore_best_weights=True),
               tf.keras.callbacks.TensorBoard(f'logs/{datetime.now()}')
               ],
)

### Training metrics
Lets inspect the training history to see how the losses and metrics behaved during training

In [ ]:
history = pd.DataFrame({'epoch': hist.epoch, **hist.history}).set_index('epoch')
history

In [ ]:
history.plot()

## Model evaluation
Lets have a look how well the model is preforming on our test dataset.

In [ ]:
predictions = model.predict(test_dataset, batch_size=BATCH_SIZE, verbose=2, use_multiprocessing=True)

In [ ]:
report = metrics.classification_report(y_true=test_labels, 
                                       y_pred=(predictions > 0.5).astype(int))
print(report)